In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
import zipfile
import shutil
import os

In [ ]:
data_dir = "path/to/dataset"
batch_size = 32
epochs = 10
output_zip = "trained_model_and_augmented.zip"

In [ ]:
def create_model(num_classes):
    """
    Example CNN model. You can customize or replace
    with a pretrained model (e.g., MobileNet, ResNet, etc.).
    """
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu',
                      input_shape=(224, 224, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
val_datagen = ImageDataGenerator(rescale=1. / 255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)
val_generator = val_datagen.flow_from_directory(
    directory=data_dir,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=True
)

num_classes = len(train_generator.class_indices)
print(f"Found {num_classes} classes: {train_generator.class_indices}")

In [ ]:
model = create_model(num_classes)
model.summary()

model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

val_loss, val_acc = model.evaluate(val_generator)
print(f"Validation accuracy: {val_acc:.2f}")

In [ ]:
model.save("trained_leaf_disease_model.h5")
print("Model saved as trained_leaf_disease_model.h5")

In [ ]:
augmented_folder = "augmented_images"
if os.path.exists(augmented_folder):
    shutil.rmtree(augmented_folder)
os.makedirs(augmented_folder, exist_ok=True)

with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zf:
    zf.write("trained_leaf_disease_model.h5")

    for root, dirs, files in os.walk(augmented_folder):
        for file in files:
            full_path = os.path.join(root, file)
            relative_path = os.path.relpath(full_path, start=".")
            zf.write(full_path, arcname=relative_path)

print(f"Zipped model and augmented images into {output_zip}")